<a href="https://colab.research.google.com/github/mdavistr-cmu/Davis-Troller_DSPN_S25/blob/main/ExerciseSubmissions/Davis-Troller_Exercise15%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/CoAxLab/Data-Explorations/blob/main/book/exercises/power-analysis-via-simulations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 15: Power analyses

This  assignment is designed to give you practice with Monte Carlo methods to conduct power analyses via simulation. You won't need to load in any data for this homework. We will, however, be using parts of the homework from last week.

---
## 1. Simulating data (1 points)


Pull your `simulate_data()` function from your last homework and add it below.

As a reminder, this function simulates the relationship between age, word reading experience, and reading comprehension skill.

`c` is reading comprehension, and `x` is word reading experience.

In [3]:
install.packages("mediation")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘rbibutils’, ‘checkmate’, ‘Rdpack’, ‘zoo’, ‘gridExtra’, ‘htmlTable’, ‘viridis’, ‘Formula’, ‘minqa’, ‘nloptr’, ‘reformulas’, ‘RcppEigen’, ‘mvtnorm’, ‘sandwich’, ‘lpSolve’, ‘Hmisc’, ‘lme4’




In [6]:
sample_size = 100 # How many children in data set?
age_lo = 80     # minimum age, in months
age_hi = 200    # maximum age, in months
beta_xa = 0.5   # amount by which experience changes for increase of one month in age
beta_x0 = -5    # amount of experience when age = 0 (not interpretable, since minimum age for this data is 80 months)
sd_x = 50       # standard dev of gaussian noise term, epsilon_x
beta_ca = 0.8   # amount that comprehension score improves for every increase of one unit in age
beta_cx = 3     # amount that comprehension score improves for every increase of one unit in reading experience
beta_c0 = 10    # comprehension score when reading experience is 0.
sd_c = 85      # standard dev of gaussian noise term, epsilon_c

simulate_data <- function(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
        age <- runif(sample_size, min = age_lo, max = age_hi)

        epsilon_x <- rnorm(sample_size, mean = 0, sd = sd_x)
        x <- beta_x0 + beta_xa * age + epsilon_x

       epsilon_c <- rnorm(sample_size, mean = 0, sd = sd_c)
       c <- beta_c0 + beta_ca * age + beta_cx * x + epsilon_c

      return(data.frame(age=age,x=x,c=c)) # it's actually bad form to have a variable named "c" in R, my bad...
}


dat <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
head(dat)

,age,x,c
,<dbl>,<dbl>,<dbl>
1,121.9412,117.77238,507.0636
2,196.3965,89.34600,464.4298
3,113.8420,150.79316,497.6648
4,143.6565,71.81062,266.2269
5,199.4166,131.47754,586.1434
6,115.1774,45.78616,404.8090


---
## 2. `run_analysis()` function (2 pts)

Last week, we looked at whether word reading experience(`x`) mediated the relation between `age` and reading comprehension (`c`).

Now we're going to use our `simulate_data()` function to conduct a power analysis. The goal is to determine how many participants we would need in order to detect both the mediated and the direct effects in this data.

*Note: We're going to pretend for the sake of simplicity that we don't have any control over the ages of the children we get (so ages are generated using `runif(sample_size, age_lo, age_hi)`, although of course this would be an unusual situation in reality.*

First, write a function, `run_analysis()`, that takes in simulated data, runs **your mediation from last week**, and returns a vector containing the ACME and ADE estimates and p-values (these are the `d0`, `d0.p`, `z0`, and `z0.p` features of the mediated model object, e.g., `fitMed$d0.p`). Print this function's output for the data we simulated previously.

In [8]:
library(mediation)

run_analysis <- function(data) {
  # Fit mediator model: x ~ age
  med_model <- lm(x ~ age, data = data)

  # Fit outcome model: comp ~ x + age
  out_model <- lm(c ~ x + age, data = data)

  # Run mediation analysis
  fitMed <- mediate(med_model, out_model, treat = "age", mediator = "x",
                    boot = TRUE, sims = 1000)

  # Extract ACME and ADE estimates and p-values
  acme <- fitMed$d0
  acme_p <- fitMed$d0.p
  ade <- fitMed$z0
  ade_p <- fitMed$z0.p

  return(c(ACME = acme, ACME_p = acme_p, ADE = ade, ADE_p = ade_p))
}


analysis_result <- run_analysis(dat)
print(analysis_result)

Running nonparametric bootstrap




     ACME    ACME_p       ADE     ADE_p 
1.6414680 0.0020000 0.5530051 0.0180000 


---
## 3. `repeat_analysis()` function (3 pts)

Next fill in the function `repeat_analysis()` below so that it simulates and analyzes data `num_simulations` times. Store the outputs from each simulation in the `simouts` matrix. Calculate and return the coverage across all the simulations run for both ACME and ADE.

In [14]:
repeat_analysis <- function(num_simulations, alpha, sample_size, age_lo, age_hi,
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
    # Initialize simouts matrix for storing each output from run_analysis()
    simouts <- matrix(rep(NA, num_simulations*4), nrow=num_simulations, ncol=4)

    # Start simulating
    for (i in 1:num_simulations) {
    simdata <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x,
                             beta_ca, beta_cx, beta_c0, sd_c)
    simouts[i, ] <- run_analysis(simdata)
  }

    # Calculate coverage for both ACME and ADE estimates using p-values in simouts
    ACME_cov = mean(simouts[, 2] < alpha, na.rm = TRUE)
    ADE_cov = mean(simouts[, 4] < alpha, na.rm = TRUE)

    return(list(ACME_cov = ACME_cov, ADE_cov = ADE_cov))
}

results <- repeat_analysis(
  num_simulations = 10,
  alpha = 0.01,
  sample_size = 100,
  age_lo = 80,
  age_hi = 200,
  beta_xa = 0.5,
  beta_x0 = -5,
  sd_x = 50,
  beta_ca = 0.8,
  beta_cx = 3,
  beta_c0 = 10,
  sd_c = 85
)

print(results)

Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap




$ACME_cov
[1] 0.8

$ADE_cov
[1] 0.8



Now run the `repeat_analysis()` function using the same parameter settings as above, for 10 simulations, with an alpha criterion of 0.01.

In [15]:
set.seed(123)


result <- repeat_analysis(
  num_simulations = 10,
  alpha = 0.01,
  sample_size = 100,
  age_lo = 80,
  age_hi = 200,
  beta_xa = 0.5,
  beta_x0 = -5,
  sd_x = 50,
  beta_ca = 0.8,
  beta_cx = 3,
  beta_c0 = 10,
  sd_c = 85
)


print(result)



Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap




$ACME_cov
[1] 1

$ADE_cov
[1] 0.6



---
## 4. Testing different sample sizes (2 pts)

Finally, do the same thing (10 simulations, alpha criterion of 0.01) but for 5 different sample sizes: 50, 75, 100, 125, 150. You can do this using `map` (as in the tutorial), or a simple `for` loop, or by calculating each individually. Up to you! This should take around 3 minutes to run.

In [16]:
sample_sizes <- c(50, 75, 100, 125, 150)

results_list <- list()

set.seed(123)

# Loop
for (s in sample_sizes) {
  cat("Running simulations for sample size:", s, "\n")

  result <- repeat_analysis(
    num_simulations = 10,
    alpha = 0.01,
    sample_size = s,
    age_lo = 80,
    age_hi = 200,
    beta_xa = 0.5,
    beta_x0 = -5,
    sd_x = 50,
    beta_ca = 0.8,
    beta_cx = 3,
    beta_c0 = 10,
    sd_c = 85
  )

  results_list[[as.character(s)]] <- result
}



Running simulations for sample size: 50 


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap




Running simulations for sample size: 75 


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap




Running simulations for sample size: 100 


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap




Running simulations for sample size: 125 


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap




Running simulations for sample size: 150 


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap


Running nonparametric bootstrap




Print your results.

In [17]:

for (s in names(results_list)) {
  cat("\nSample size:", s, "\n")
  print(results_list[[s]])
}



Sample size: 50 
$ACME_cov
[1] 0.6

$ADE_cov
[1] 0.3


Sample size: 75 
$ACME_cov
[1] 0.7

$ADE_cov
[1] 0.4


Sample size: 100 
$ACME_cov
[1] 0.8

$ADE_cov
[1] 0.6


Sample size: 125 
$ACME_cov
[1] 1

$ADE_cov
[1] 0.9


Sample size: 150 
$ACME_cov
[1] 1

$ADE_cov
[1] 0.9



## 5. Reflection (2 pts)

If this were a real power analysis, we'd want to run more simulations per sample size (to get a more precise estimate of power) and we may also want to test out some other values of the parameters we used to simulate our data. However, what would you conclude just based on the results above?

> We reach 0.80 ACME at 100 subjects and 1.00 by 125 subjects which means 125 is when we reach full power to detect the mediation effect, but 100 is sufficient. ADE grows more slowly and thus we need at least 125 participants to meet the conventional 0.80 power threshold. Therefore, overall we would need 125 participants to detect a mediation effect and direct effect with high power.




**Given** how we generated the data, why was the direct effect harder to detect than the mediated effect?
> The direct effect was harder to detect because it was smaller in magnitude than the mediated effect, and its path has more unexplained variance. In contrast, the mediated effect benefited from two potentially strong paths (age → x and x → c), making it more detectable, even at smaller sample sizes.

**DUE:** 5pm EST, April 3, 2024

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here.
> *Someone's Name*